# Background subtraction using top-hat in scikit-image and pyclesperanto
This notebook compares different implementations of a background subtraction method.

**Note:** benchmarking results vary heavily depending on image size, kernel size, used operations, parameters and used hardware. Use this notebook to adapt it to your use-case scenario and benchmark on your target hardware. If you have different scenarios or use-cases, you are very welcome to submit your notebook as pull-request!

In [1]:
import pyclesperanto_prototype as cle
from skimage import morphology
import time

# to measure kernel execution duration properly, we need to set this flag. It will slow down exection of workflows a bit though
cle.set_wait_for_kernel_finish(True)

# selet a GPU with the following in the name. This will fallback to any other GPU if none with this name is found
cle.select_device('RTX')

<Intel(R) UHD Graphics on Platform: Intel(R) OpenCL HD Graphics (1 refs)>

In [2]:
radius = 10
disk_kernel = morphology.disk(radius)
square_kernel = morphology.square(radius)

In [3]:
# test data
import numpy as np

test_image = np.random.random([1500, 2000])

In [4]:
# top-hat (disk) with scikit-image
result_image = None

for i in range(0, 5):
    start_time = time.time()
    result_image = morphology.white_tophat(test_image, selem=disk_kernel)
    print("skimage top-hat disk duration: " + str(time.time() - start_time))

skimage top-hat disk duration: 2.4475488662719727
skimage top-hat disk duration: 3.3568480014801025
skimage top-hat disk duration: 7.346466302871704
skimage top-hat disk duration: 7.228134632110596
skimage top-hat disk duration: 7.357930421829224


In [5]:
# top-hat (square) with scikit-image
result_image = None

for i in range(0, 5):
    start_time = time.time()
    result_image = morphology.white_tophat(test_image, selem=square_kernel)
    print("skimage top-hat square duration: " + str(time.time() - start_time))

skimage top-hat square duration: 0.38732314109802246
skimage top-hat square duration: 0.38617515563964844
skimage top-hat square duration: 0.41457080841064453
skimage top-hat square duration: 0.4019930362701416
skimage top-hat square duration: 0.40062451362609863


In [6]:
# top-hat (disk) with pyclesperanto
result_image = None

test_image_gpu = cle.push(test_image)

for i in range(0, 5):
    start_time = time.time()
    result_image = cle.top_hat_sphere(test_image_gpu, result_image, radius_x=radius, radius_y=radius)
    print("pyclesperanto top-hat-shere duration: " + str(time.time() - start_time))

pyclesperanto top-hat-shere duration: 0.29504895210266113
pyclesperanto top-hat-shere duration: 0.2683713436126709
pyclesperanto top-hat-shere duration: 0.28290772438049316
pyclesperanto top-hat-shere duration: 0.2635986804962158
pyclesperanto top-hat-shere duration: 0.2735409736633301


In [7]:
# top-hat (square) using pyclesperanto
result_image = None

test_image_gpu = cle.push(test_image)

for i in range(0, 5):
    start_time = time.time()
    result_image = cle.top_hat_box(test_image_gpu, result_image, radius_x=radius, radius_y=radius)
    print("pyclesperanto top-hat-box duration: " + str(time.time() - start_time))

pyclesperanto top-hat-box duration: 0.0683290958404541
pyclesperanto top-hat-box duration: 0.046486854553222656
pyclesperanto top-hat-box duration: 0.04851675033569336
pyclesperanto top-hat-box duration: 0.0616755485534668
pyclesperanto top-hat-box duration: 0.04777789115905762
